In [1]:
!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
sys.path.append('./')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle
root_dir = './'

In [2]:
# HIDE OUTPUT
!wget http://www.atarimania.com/roms/Roms.rar 
!unrar x -o+ /content/Roms.rar >/dev/nul
!python -m atari_py.import_roms /content/ROMS >/dev/nul

'wget' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
The system cannot find the path specified.


In [3]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("Breakout-v0")

C:\Users\kilorad\Anaconda3\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4` with the environment ID `Breakout-v4`.
  logger.warn(


Error: We're Unable to find the game "Breakout". Note: Gym no longer distributes ROMs. If you own a license to use the necessary ROMs for research purposes you can download them via `pip install gym[accept-rom-license]`. Otherwise, you should try importing "Breakout" via the command `ale-import-roms`. If you believe this is a mistake perhaps your copy of "Breakout" is unsupported. To check if this is the case try providing the environment variable `PYTHONWARNINGS=default::ImportWarning:ale_py.roms`. For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management

In [ ]:
if 0:
  import tensorflow as tf
  import tensorflow.keras as keras
  from tensorflow.keras import layers
  from tensorflow.keras.models import Sequential, Model
  from tensorflow.keras.layers import ConvLSTM2D,Conv3D,Conv2D,TimeDistributed,Flatten, Reshape, Input
  from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Permute, MaxPooling2D, MaxPooling3D, UpSampling2D
  from tensorflow.keras.layers import Dense, Dropout
  from keras.layers import LeakyReLU
  from tensorflow.keras.layers import GRU, LSTM
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras.layers import Activation

  from tensorflow.keras.preprocessing import image
  from keras import backend as K
  def reset_keras():
    K.clear_session()
    gc.collect()
  try:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
  except Exception:
    print('GPU error')
  state_size = (224,224,3)
  mobnet = tf.keras.applications.mobilenet.MobileNet(
      input_shape=state_size, alpha=1.0, depth_multiplier=1, dropout=0.001,
      include_top=True, weights='imagenet', input_tensor=None, pooling=None,
      classes=1000, classifier_activation='softmax'
  )
  mobnet = Model(inputs=mobnet.input, outputs=mobnet.get_layer('global_average_pooling2d').output)
  def resize_arr_img(x,size):
    x = image.array_to_img(x)
    x = x.resize(size)
    return np.array(x)

In [ ]:
if 0:
  def encode_mobnet(X):
    #каждую картинку порубить на 4 части
    count_squares = 2
    sz = np.shape(X)
    x_lst = []
    for i in range(count_squares):
      for j in range(count_squares):
        x = X[:,int(i*sz[1]/count_squares):int((i+1)*sz[1]/count_squares), int(j*sz[2]/count_squares):int((j+1)*sz[2]/count_squares)]
        x = np.array([resize_arr_img(x[i],(224,224)) for i in range(np.shape(x)[0])])
        #y = mobnet.predict(x)
        x_lst.append(x)
        #y_lst.append(y)
    #а потом на 9
    count_squares = 3
    for i in range(count_squares):
      for j in range(count_squares):
        x = X[:,int(i*sz[1]/count_squares):int((i+1)*sz[1]/count_squares), int(j*sz[2]/count_squares):int((j+1)*sz[2]/count_squares)]
        x = np.array([resize_arr_img(x[i],(224,224)) for i in range(np.shape(x)[0])])
        #y = mobnet.predict(x)
        x_lst.append(x)
        #y_lst.append(y)
    x_lst = np.array(x_lst)[:,0]
    y = mobnet.predict(x_lst)
    y_lst = np.reshape(y,[sz[0],13*1024])
    #print('y_lst',np.shape(y_lst),'must be 2000x4098')
    return y_lst

In [ ]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 7
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 3
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 2
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [ ]:
n_actions = 4
embed_size = 62#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open(root_dir+f'./genoms/best_genoms_breakout.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

In [ ]:
def test_atary(genom, draw=False,game_name="Breakout-v4"):
  if draw:
    globals()['video'] = []

  seed=1
  np.random.seed(seed)
  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0
  for i in range(150):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
      t=pd.Timestamp.now()
      #state = encode_mobnet(np.array([state]))
      state = encode_simple(state)
      reward_sum += reward
      if np.shape(state)[0]>1:
          shp = np.shape(state)
          state = np.reshape(state,[1,shp[0]])
      out_tape = controller.act(state,reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum

In [ ]:
def multy_test(genom):
  game_name = "Breakout-v0"
  #один энвайронмент, разные сиды
  q_arr = []
  for seed in range(10):
      q_arr.append(test_atary(genom, draw=False,game_name=game_name))
  return np.mean(q_arr)

In [ ]:
%%time
q=test_atary(genom,game_name="Breakout-v0", draw=False)
print('q=',q)

In [ ]:
%%time
q=multy_test(genom)
print('q=',q)

In [ ]:
%%time
q=test_atary(genom, draw=True)
print('q=',q)

In [ ]:
video[0].save(
    root_dir+'./out_videos/breakout_neuroevol.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [ ]:
print(pd.Timestamp.now())

In [ ]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=2)
try:
    with open(root_dir+f'best_genoms.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass
opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'best_genoms.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')